<a href="https://colab.research.google.com/github/Dromediansk/job-preparation-crewai/blob/main/Crew_AI_Interview_Prep_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install 'crewai[tools]' -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.0/677.0 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.6/380.6 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 8.8 MB/s e

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/CrewAI Interview Prep Part 2

/content/drive/MyDrive/Colab Notebooks/CrewAI Interview Prep Part 2


In [ ]:
# Retrieve API keys and store them as Environment variables
from google.colab import userdata
openai_api_key = userdata.get('AI_AGENTS_OPENAI')
serper_api_key = userdata.get('SERPER_API_KEY')

import os
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['SERPER_API_KEY'] = serper_api_key


In [ ]:
# Import libraries
from crewai import Agent, Crew, Process, Task
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
llm = ChatOpenAI(model = 'gpt-4o', temperature=0.5)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


# User Inputs

In [ ]:
# Call for inputs: Inverviewer, company, job position, job description:
interviewer = input('Enter the name of the interviewer: ')
company = input('Enter the name of the company: ')
job_position = input('Enter the job position: ')
job_description = input('Enter the job description: ')

Enter the name of the interviewer: Richard Roško
Enter the name of the company: Sudolabs
Enter the job position: AI Engineer
Enter the job description: About the job Type of contract: full-time or trade license  Place of work: Kosice or remote  Start date: let us know what time suits you best  Compensation: from €20/hour up to €45/hour  ‍  You’ll be part of a small team of engineers, data scientists, product managers, and designers where rapid prototyping and fast delivery are the norm. We don’t just follow AI trends—we translate cutting-edge research into real-world products. One day, you’ll be experimenting with breakthroughs that seemed impossible two weeks ago; the next, you’ll be preparing for what’s just around the corner (sometimes quite literally waiting for OpenAI’s next release to unlock new capabilities 😅). If you enjoy solving complex AI challenges and turning research into real products, we'd love to hear from you.  As An AI Engineer, We Expect You To    Take ownership of 

# Research Crew

## Research AI Agent

In [ ]:
# Create the Research AI Agent
research_agent = Agent(
    role = "Research Agent",
    goal = "Conduct in-depth research",
    backstory = """
    As a Research Specialist, your mission is to uncover detailed and relevant information.
    The ultimate goal is to meet the expectations of the task
    """,
    llm = llm,
    tools = [ScrapeWebsiteTool(), SerperDevTool()]
)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


### Research Tasks

In [ ]:
# Define the task about researching the company
research_company_task = Task(
    description = f"""
    Research about {company}, their position in the sector and recent industry trends.
    Focus on recent news and development from last 12 months
    """,
    expected_output = f""" A comprehensive report in Markdown with bullet points about {company}""",
    agent = research_agent
)

In [ ]:
# Research Interviewer Task
research_person_task = Task(
    description = f"Research about {interviewer} with a focus on their professional career",
    expected_output= f""" A comprehensive report in Markdown with bullet points about {interviewer}""",
    agent = research_agent
)

### AI Coach + Interview Prep

In [ ]:
# Build the AI interview coach agent
coach_agent = Agent(
    role = "Interview Coach",
    goal = f"I help the user to prepare for the job interview for job {job_position}",
    backstory = f"You are an expert in job interviews in companies like {company}",
    llm = llm
    )

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
# Prepare the list of questions
define_questions_task = Task(
    description = f"""
    Define a list of questions based on the company {company}, job description {job_description} and interviewer {interviewer}
""",
    expected_output= f"""A list of 20 questions in Markdown format that cover: Culture and Team fit, Job position fit, Background and ways of working, Growth Mindset""",
    agent = coach_agent,
    context = [research_company_task, research_person_task]
)

## Assemble the Crew

In [ ]:
# Assembling the crew
crew = Crew(
    agents = [research_agent, coach_agent],
    tasks = [research_company_task, research_person_task, define_questions_task],
    verbose = True,
    process = Process.sequential
)
result = crew.kickoff({"topic": "Write a list of questions to prepare user for the interview"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 6e5cbce3-82f7-4728-9173-a6db24b6fc62                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research Agent                                                                                          │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Research about Sudolabs, their position in the sector and recent industry trends.                          │
│      Focus on recent news and development from last 12 months                                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

/usr/local/lib/python3.12/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: 
datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects 
to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)

...

Output()

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Interview Coach                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Here is a list of 20 questions in Markdown format that cover Culture and Team fit, Job position fit,           │
│  Background and ways of working, and Growth Mindset for the AI Engineer position at Sudolabs:                   │
│                                                                                                                 │
│  ```markdown                                                                                                    │
│  ## Culture and Team Fit                                                                                        │
│  1. Can you describe the company culture at Sudolabs and how it supports innovation and creativity?             │
│  2. How does Sudolabs encourage collaboration among its small teams of engineers, data scientists, product      │
│  managers, and designers?                                                                                       │
│  3. What are some examples of how Sudolabs has supported team members' personal and professional growth?        │
│  4. How does Sudolabs ensure a healthy work-life balance for its employees, especially those working remotely?  │
│  5. What values are most important to Sudolabs, and how do they influence the way the team operates?            │
│                                                                                                                 │
│  ## Job Position Fit                                                                                            │
│  6. What are the primary responsibilities of an AI Engineer at Sudolabs, and how does this role contribute to   │
│  the company's goals?                                                                                           │
│  7. Can you provide examples of AI-driven projects that an AI Engineer at Sudolabs might work on?               │
│  8. How does Sudolabs balance rapid prototyping with ensuring high-quality product delivery?                    │
│  9. What are the key challenges an AI Engineer might face at Sudolabs, and how can they overcome them?          │
│  10. How does Sudolabs measure the success of an AI Engineer in this role?                                      │
│                                                                                                                 │
│  ## Background and Ways of Working                                                                              │
│  11. What technical skills are most critical for the AI Engineer role at Sudolabs, and how are they applied in  │
│  day-to-day tasks?                                                                                              │
│  12. How does Sudolabs support continuous learning and staying updated with the latest AI trends and            │
│  technologies?                                                                                                  │
│  13. Can you describe the typical workflow for an AI project at Sudolabs, from conception to production?        │
│  14. How important is it for an AI Engineer at Sudolabs to have experience with full-stack development, and     │
│  why?                                                                                                           │
│  15. How does Sudolabs approach risk management and client expectation management in AI-related projects?       │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 2b7c1094-15da-4d8e-959f-64b327f08d9b                                                                     │
│  Agent: Interview Coach                                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 6e5cbce3-82f7-4728-9173-a6db24b6fc62                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Here is a list of 20 questions in Markdown format that cover Culture and Team fit, Job position  │
│  fit, Background and ways of working, and Growth Mindset for the AI Engineer position at Sudolabs:              │
│                                                                                                                 │
│  ```markdown                                                                                                    │
│  ## Culture and Team Fit                                                                                        │
│  1. Can you describe the company culture at Sudolabs and how it supports innovation and creativity?             │
│  2. How does Sudolabs encourage collaboration among its small teams of engineers, data scientists, product      │
│  managers, and designers?                                                                                       │
│  3. What are some examples of how Sudolabs has supported team members' personal and professional growth?        │
│  4. How does Sudolabs ensure a healthy work-life balance for its employees, especially those working remotely?  │
│  5. What values are most important to Sudolabs, and how do they influence the way the team operates?            │
│                                                                                                                 │
│  ## Job Position Fit                                                                                            │
│  6. What are the primary responsibilities of an AI Engineer at Sudolabs, and how does this role contribute to   │
│  the company's goals?                                                                                           │
│  7. Can you provide examples of AI-driven projects that an AI Engineer at Sudolabs might work on?               │
│  8. How does Sudolabs balance rapid prototyping with ensuring high-quality product delivery?                    │
│  9. What are the key challenges an AI Engineer might face at Sudolabs, and how can they overcome them?          │
│  10. How does Sudolabs measure the success of an AI Engineer in this role?                                      │
│                                                                                                                 │
│  ## Background and Ways of Working                                                                              │
│  11. What technical skills are most critical for the AI Engineer role at Sudolabs, and how are they applied in  │
│  day-to-day tasks?                                                                                              │
│  12. How does Sudolabs support continuous learning and staying updated with the latest AI trends and            │
│  technologies?                                                                                                  │
│  13. Can you describe the typical workflow for an AI project at Sudolabs, from conception to production?        │
│  14. How important is it for an AI Engineer at Sudolabs to have experience with full-stack development, and     │
│  why?                                                                                                           │
│  15. How does Sudolabs approach risk management and cl

# Exporting

In [ ]:
# Check the crew outcome
display(Markdown(result.tasks_output[0].raw))

The above report provides a comprehensive overview of Sudolabs, detailing their position in the sector, recent achievements, industry events, company metrics, and future outlook.

In [ ]:
# Export each of the tasks output
# Exportin the company report
file_name = f"{company}_report.txt"

# Open the file in write mode and write the content
with open(file_name, 'w') as file:
    file.write(result.tasks_output[0].raw)

# Export the interview questions
file_name = f"{job_position}.questions.txt"

# Open the file in write mode and write the content
with open(file_name, 'w') as file:
    file.write(result.tasks_output[2].raw)



# Interview Prep

In [ ]:
# Import the questions from directory
with open(f"{job_position}.questions.txt", 'r') as file:
    questions = file.read()

In [ ]:
# AI Agent that ask the question
interviewer_agent = Agent(
    role = "Interviewer Agent",
    goal = f"Ask all questions from {questions} to the user",
    backstory = f"You are an expert in job interviews in companies like {company}",
    llm = llm
)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
# Interview Task
interview_prep_task = Task(
    description = f"Ask a question from {questions} to the user",
    expected_output= f"Ask only one question at a time",
    agent = interviewer_agent,
    human_input = True
)

In [ ]:
# Feedback Task
feedback_task = Task(
    description = f"Give feedback to the user on its answer",
    expected_output= f"Markdown answer what was good, could be improved and how to take it to the next level",
    agent = coach_agent,
    context = [interview_prep_task]
)

## Interview Prep Crew

In [ ]:
# Assemble the interview Prep Crew
crew = Crew(
    agents = [interviewer_agent, coach_agent],
    tasks = [interview_prep_task, feedback_task],
    verbose = True,
    process = Process.sequential
)
result = crew.kickoff({"topic": f"Ask user all questions from {questions}"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: f9834eec-66dd-45e5-b29a-36f87bfed050                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Interviewer Agent                                                                                       │
│                                                                                                                 │
│  Task: Ask a question from Here is a list of 20 questions in Markdown format that cover Culture and Team fit,   │
│  Job position fit, Background and ways of working, and Growth Mindset for an AI Engineer position at Sudolabs,  │
│  considering the provided context:                                                                              │
│                                                                                                                 │
│  ```markdown                                                                                                    │
│  # Culture and Team Fit                                                                                         │
│  1. Can you describe the company culture at Sudolabs and how it supports innovation and creativity?             │
│  2. How does Sudolabs ensure a collaborative environment among engineers, data scientists, and designers?       │
│  3. What values are most important to the team at Sudolabs, and how are they reflected in daily work?           │
│  4. How does Sudolabs support remote employees to feel connected with the team?                                 │
│  5. Can you share an example of a recent team-building activity or company retreat?                             │
│                                                                                                                 │
│  # Job Position Fit                                                                                             │
│  6. What are the key challenges an AI Engineer might face at Sudolabs and how can they be overcome?             │
│  7. How does Sudolabs define success in the AI Engineer role?                                                   │
│  8. What kind of AI projects has Sudolabs recently completed, and what role did engineers play in them?         │
│  9. How is feedback given and received within the engineering team?                                             │
│  10. Can you describe a typical day for an AI Engineer at Sudolabs?                                             │
│                                                                                                                 │
│  # Background and Ways of Working                                                                               │
│  11. What kind of professional development opportunities are available for AI Engineers at Sudolabs?            │
│  12. How does Sudolabs support engineers in staying updated with the latest AI trends and technologies?         │
│  13. What tools and technologies does Sudolabs primarily use for AI development?                                │
│  14. How does the team approach rapid prototyping and fast delivery of AI solutions?                            │
│  15. Can you describe the process of engaging with clients to understand their AI needs?                        │
│                                                                                                                 │
│  # Growth Mindset                                                                                               │
│  16. How does Sudolabs encourage continuous learning and knowledge sharing among its employees?                 │
│  17. What opportunities exist for AI Engineers to contribute to the internal knowledge base or mentor others?   │
│  18. How does Sudolabs handle failures or setbacks in AI projects, and what is the learning process from them?  │
│  19. Can you provide examples of how AI Engineers at Su

Output()

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Interviewer Agent                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Can you describe the company culture at Sudolabs and how it supports innovation and creativity?                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

 ## Final Result: Can you describe the company culture at Sudolabs and how it supports innovation and creativity?
 

=====
## HUMAN FEEDBACK: Provide feedback on the Final Result and Agent's actions.
Please follow these guidelines:
 - If you are happy with the result, simply hit Enter without typing anything.
 - Otherwise, provide specific improvement requests.
 - You can provide multiple rounds of feedback until satisfied.
=====



KeyboardInterrupt: Interrupted by user